In [1]:
import wfdb as wf
from wfdb import processing

import neurokit2 as ECG
import librosa as mi


import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft
import numpy as np
import os

In [2]:
def Feature_CC(segment):
    try:
        win = signal.windows.hamming(len(segment))
        win_segment = np.multiply(segment, win)

        fft_c = fft(win_segment,len(win_segment))
        log_fft_c = np.log(np.abs(fft_c))
        cc= ifft(log_fft_c)

        return np.real(cc[:12])
    except IndexError:
        print("Index_error")

def Feature_Entropy(segment):


    counts, _ = np.histogram(segment, bins=5)
    P_x = counts/sum(counts)

    entropy = sum(np.multiply(P_x,np.log2(P_x)))

    return entropy
        


def Feature_ZCR(segment):
    return np.sum(mi.zero_crossings(segment).astype(float))

def ConvertArray2String(B):
    Feature = ""
    for i in B:
        Feature = Feature + ',' +str(i)
    return Feature


In [3]:
os.chdir('..')
dir = os.getcwd()

In [4]:
os.chdir(dir)
os.chdir(dir + '/DATASET/ECG-ID')
IDs = os.listdir()
print(IDs)

# Channel number
N = 0

f = open(dir + '/Features' +'/'+ f"Feature_ECG_ID_Channel_{N}.csv", "a")
f.write("ID,CC_1,CC_2,CC_3,CC_4,CC_5,CC_6,CC_7,CC_8,CC_9,CC_10,CC_11,CC_12,ENTROPY,ZCR,PERSON,TARGET\n")

k=0
for ID in IDs:
    print(ID)
    os.chdir(  os.getcwd()+ '/'+ ID)
    Audio = os.listdir()
    Audio_list = [k for k in Audio if '.dat' in k]



    for A in Audio_list:
        
        print(A)
        signal_array, fields = wf.rdsamp( os.getcwd() +  "/" + A.replace(".dat", ""))
        sig = signal_array[:,N]

        # For single channel
        sos = signal.butter(6, [2, 50], btype='bandpass', fs=fields['fs'], analog= False)
        filtered = signal.filtfilt(sos[0], sos[1], sig)

        try:
            _, rpeaks = ECG.ecg_peaks(filtered, sampling_rate=fields['fs'])
            _, waves_peak = ECG.ecg_delineate(filtered, rpeaks, sampling_rate=1000, method="peak")
            t_peaks_ind = waves_peak['ECG_T_Peaks']
        except IndexError:
            print('Skipping' + A)
            break
        except TypeError:
            print('Skipping' + A)
            break
        except ValueError:
            print('Skipping' + A)
            break
        

        for n in range(len(t_peaks_ind)-1):

            try:
                segment = filtered[int(t_peaks_ind[n]):int(t_peaks_ind[n+1])]
                CC = Feature_CC(segment)
                Entropy = Feature_Entropy(segment) 
                ZCR = Feature_ZCR(segment)

                feature_string = ID + '_' + A.replace('.dat','') +'_' + str(n) + ConvertArray2String(CC) + ',' + str(Entropy) + ',' +  str(ZCR) + ',' + ID.replace('Person_','') + ',' + str(k) + '\n'
                f.write(feature_string)
            except ValueError:
                print('Skipping a segment')


    os.chdir(dir + '/DATASET/ECG-ID')
    k+=1

f.close()

['Person_58', 'Person_19', 'Person_75', 'Person_78', 'Person_45', 'Person_41', 'Person_01', 'Person_23', 'Person_32', 'Person_46', 'Person_14', 'Person_16', 'Person_28', 'Person_18', 'Person_21', 'Person_82', 'Person_51', 'Person_54', 'Person_87', 'Person_29', 'Person_62', 'Person_40', 'Person_70', 'Person_74', 'Person_49', 'Person_85', 'Person_69', 'Person_50', 'Person_76', 'Person_86', 'Person_13', 'Person_44', 'Person_24', 'Person_27', 'Person_37', 'Person_31', 'Person_48', 'Person_53', 'Person_56', 'Person_64', 'Person_89', 'Person_80', 'Person_77', 'Person_39', 'Person_57', 'Person_12', 'Person_83', 'Person_88', 'Person_10', 'Person_65', 'Person_26', 'Person_47', 'Person_61', 'Person_22', 'Person_38', 'Person_72', 'Person_73', 'Person_11', 'Person_04', 'Person_02', 'Person_34', 'Person_52', 'Person_68', 'Person_90', 'Person_67', 'Person_84', 'Person_59', 'Person_42', 'Person_07', 'Person_17', 'Person_43', 'Person_71', 'Person_55', 'Person_63', 'Person_08', 'Person_03', 'Person_15'

KeyboardInterrupt: 

In [ ]:
os.chdir(dir)
os.chdir(dir + '/DATASET/ECG-ID')
IDs = os.listdir()
print(IDs)

# Channel number
N = 1

f = open(dir + '/Features' +'/'+ f"Feature_ECG_ID_Channel_{N}.csv", "a")
f.write("ID,CC_1,CC_2,CC_3,CC_4,CC_5,CC_6,CC_7,CC_8,CC_9,CC_10,CC_11,CC_12,ENTROPY,ZCR,PERSON,TARGET\n")

for ID in IDs:
    print(ID)
    os.chdir(  os.getcwd()+ '/'+ ID)
    Audio = os.listdir()
    Audio_list = [k for k in Audio if '.dat' in k]



    for A in Audio_list:
        
        print(A)
        signal_array, fields = wf.rdsamp( os.getcwd() +  "/" + A.replace(".dat", ""))
        sig = signal_array[:,N]

        # For single channel
        sos = signal.butter(6, [2, 50], btype='bandpass', fs=fields['fs'], analog= False)
        filtered = signal.filtfilt(sos[0], sos[1], sig)

        try:
            _, rpeaks = ECG.ecg_peaks(filtered, sampling_rate=fields['fs'])
            _, waves_peak = ECG.ecg_delineate(filtered, rpeaks, sampling_rate=1000, method="peak")
            t_peaks_ind = waves_peak['ECG_T_Peaks']
        except IndexError:
            print('Skipping' + A)
            break
        except TypeError:
            print('Skipping' + A)
            break
        except ValueError:
            print('Skipping' + A)
            break
        

        for n in range(len(t_peaks_ind)-1):

            try:
                segment = filtered[int(t_peaks_ind[n]):int(t_peaks_ind[n+1])]
                CC = Feature_CC(segment)
                Entropy = Feature_Entropy(segment) 
                ZCR = Feature_ZCR(segment)

                feature_string = ID + '_' + A.replace('.dat','') +'_' + str(n) + ConvertArray2String(CC) + ',' + str(Entropy) + ',' +  str(ZCR) + ',' + ID.replace('Person_','') + ',' + str(k) + '\n'
                f.write(feature_string)
            except ValueError:
                print('Skipping a segment')

    os.chdir(dir + '/DATASET/ECG-ID')

f.close()